# Pandas vs Pyspark

## Pandas
+ Pandas run operations on a single machine whereas PySpark runs on multiple machines
+ Pandas DataFrame’s are mutable and are not lazy, statistical functions are applied on each column by default, you don’t have to explicitly specify on what column you wanted to apply the statistical functions.Pandas add an index sequence number to every data frame


## PySpark

+ PySpark is a Spark library written in Python to run Python applications in Apache Spark ecosystem . 
+ We can run applications parallelly on a distributed cluster (multiple nodes) or on a single node for development.
+ PySpark API uses Py4J. Py4J is a Java library that is integrated within PySpark and allows python to dynamically interface with JVM objects. To run PySpark you also need Java to be installed along with Python, and Apache Spark

## Download and install Spark , Java , findspark

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

## Setup environment variables

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [16]:
#List required environment variables
for k, v in sorted(os.environ.items()):
    if k in ['JAVA_HOME','SPARK_HOME','PYTHONPATH','PYSPARK_PYTHON']:
      print(k+':', v)

JAVA_HOME: /usr/lib/jvm/java-8-openjdk-amd64
PYSPARK_PYTHON: /usr/bin/python3
PYTHONPATH: /env/python
SPARK_HOME: /content/spark-2.3.1-bin-hadoop2.7


# Initiate a spark session

PySpark isn't on sys.path by default

findspark adds pyspark to sys.path at runtime. 

Same can be achieved by exporting PYSPARK_PYTHON variable manually

In [8]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [13]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading Wine Quality Data

In [18]:
!wget http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip
!unzip winequality.zip

--2022-05-06 02:08:58--  http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip
Resolving www3.dsi.uminho.pt (www3.dsi.uminho.pt)... 193.136.11.133
Connecting to www3.dsi.uminho.pt (www3.dsi.uminho.pt)|193.136.11.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96005 (94K) [application/x-zip-compressed]
Saving to: ‘winequality.zip’

winequality.zip     100%[===================>]  93.75K   342KB/s    in 0.3s    

2022-05-06 02:08:59 (342 KB/s) - ‘winequality.zip’ saved [96005/96005]

Archive:  winequality.zip
  inflating: winequality/winequality-names.txt  
  inflating: winequality/winequality-names.txt.bak  
  inflating: winequality/winequality-red.csv  
  inflating: winequality/winequality-white.csv  


In [ ]:
!ls winequality/*

winequality/winequality-names.txt      winequality/winequality-red.csv
winequality/winequality-names.txt.bak  winequality/winequality-white.csv


## Load files with Pandas

In [19]:
import pandas as pd
df_white = pd.read_csv('winequality/winequality-white.csv',header=0, delimiter=";")
df_white.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


## Load Files with PySpark

In [185]:
from pyspark.sql.functions import col,lit
df_red = spark.read.csv('winequality/winequality-red.csv',header=True,sep=";")
df_red.show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+-----------

## Pandas - Displaying data

In [21]:
# Displays top 5 rows by default. Pass a number as parameter to display different count
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


## Pyspark - Displaying data

In [26]:
# Returns a list. First row by default. Pass a number as parameter
df_red.head()

Row(fixed acidity='7.4', volatile acidity='0.7', citric acid='0', residual sugar='1.9', chlorides='0.076', free sulfur dioxide='11', total sulfur dioxide='34', density='0.9978', pH='3.51', sulphates='0.56', alcohol='9.4', quality='5')

In [27]:
# Returns a list of Row. The number parameter is mandatory for take action
df_red.take(3)

[Row(fixed acidity='7.4', volatile acidity='0.7', citric acid='0', residual sugar='1.9', chlorides='0.076', free sulfur dioxide='11', total sulfur dioxide='34', density='0.9978', pH='3.51', sulphates='0.56', alcohol='9.4', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.88', citric acid='0', residual sugar='2.6', chlorides='0.098', free sulfur dioxide='25', total sulfur dioxide='67', density='0.9968', pH='3.2', sulphates='0.68', alcohol='9.8', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.76', citric acid='0.04', residual sugar='2.3', chlorides='0.092', free sulfur dioxide='15', total sulfur dioxide='54', density='0.997', pH='3.26', sulphates='0.65', alcohol='9.8', quality='5')]

In [29]:
# Returns a dataframe. The parameter is mandatory.
df_red.limit(3)

DataFrame[fixed acidity: string, volatile acidity: string, citric acid: string, residual sugar: string, chlorides: string, free sulfur dioxide: string, total sulfur dioxide: string, density: string, pH: string, sulphates: string, alcohol: string, quality: string]

In [30]:
# Returns formatted data. Comparable to Pandas' head() action
df_red.show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+-----------

In [ ]:
# Returns a list. Be very careful while executing this for large dataset as it brings all data to the Driver node
df_red.collect()

## Pandas : Selecting columns from dataframe

In [41]:
# Display all the column names. Return an index object
df_white.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [34]:
# Get column data with . operator
df_white.density

0       1.00100
1       0.99400
2       0.99510
3       0.99560
4       0.99560
         ...   
4893    0.99114
4894    0.99490
4895    0.99254
4896    0.98869
4897    0.98941
Name: density, Length: 4898, dtype: float64

In [35]:
# Get column data using []. This is required when the column name has space 
df_white['fixed acidity']

0       7.0
1       6.3
2       8.1
3       7.2
4       7.2
       ... 
4893    6.2
4894    6.6
4895    6.5
4896    5.5
4897    6.0
Name: fixed acidity, Length: 4898, dtype: float64

In [37]:
# Get multiple columns data using []. Pass a list of column names 
df_white[['fixed acidity','volatile acidity','citric acid']]

,fixed acidity,volatile acidity,citric acid
0,7.0,0.27,0.36
1,6.3,0.30,0.34
2,8.1,0.28,0.40
3,7.2,0.23,0.32
4,7.2,0.23,0.32
...,...,...,...
4893,6.2,0.21,0.29
4894,6.6,0.32,0.36
4895,6.5,0.24,0.19
4896,5.5,0.29,0.30


In [75]:
# Select column by index/position. 
# Displays top 3 rows and column at index 1 (volatile acidity in this case)
df_white.iloc[:3,1]

0    0.27
1    0.30
2    0.28
Name: volatile acidity, dtype: float64

## Pyspark : Selecting columns from dataframe

In [38]:
# Display all the column names . Returns a list
df_red.columns

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [49]:
# Display single column
df_red.select('density').show(4)

+-------+
|density|
+-------+
| 0.9978|
| 0.9968|
|  0.997|
|  0.998|
+-------+
only showing top 4 rows



In [47]:
# Display multiple columns
df_red.select('density','sulphates','alcohol').show(4)

+-------+---------+-------+
|density|sulphates|alcohol|
+-------+---------+-------+
| 0.9978|     0.56|    9.4|
| 0.9968|     0.68|    9.8|
|  0.997|     0.65|    9.8|
|  0.998|     0.58|    9.8|
+-------+---------+-------+
only showing top 4 rows



In [56]:
# Select all columns
df_red.select("*").show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+-----------

In [55]:
#Using col function, use alias() to get alias name. This requires import of col function => from pyspark.sql.functions import col
df_red.select(col("fixed acidity").alias("fixed_acidity"),col("alcohol")).show(4)

+-------------+-------+
|fixed_acidity|alcohol|
+-------------+-------+
|          7.4|    9.4|
|          7.8|    9.8|
|          7.8|    9.8|
|         11.2|    9.8|
+-------------+-------+
only showing top 4 rows



In [59]:
# Select column by position
# Selects 4th column (index starts from zero)
df_red.select(df_red.columns[3]).show(4)

+--------------+
|residual sugar|
+--------------+
|           1.9|
|           2.6|
|           2.3|
|           1.9|
+--------------+
only showing top 4 rows



In [66]:
# Select column by position
# Selects columns from index 2 to 6
df_red.select(df_red.columns[2:6]).show(4)

+-----------+--------------+---------+-------------------+
|citric acid|residual sugar|chlorides|free sulfur dioxide|
+-----------+--------------+---------+-------------------+
|          0|           1.9|    0.076|                 11|
|          0|           2.6|    0.098|                 25|
|       0.04|           2.3|    0.092|                 15|
|       0.56|           1.9|    0.075|                 17|
+-----------+--------------+---------+-------------------+
only showing top 4 rows



In [97]:
# Select column by regular expression

df_red.select(df_red.colRegex("`.*acidity*`")).show(4)

+-------------+----------------+
|fixed acidity|volatile acidity|
+-------------+----------------+
|          7.4|             0.7|
|          7.8|            0.88|
|          7.8|            0.76|
|         11.2|            0.28|
+-------------+----------------+
only showing top 4 rows



## Pandas Add a new column 

In [127]:
# To Add a new column that is a produt of existing column. 
# As this operation has reassignment, the df_white dataframe will have the new column
df_white['ph_alcohol'] = df_white['alcohol']*df_white['pH']
df_white.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ph_alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,26.400
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,31.350
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,32.926


In [162]:
# Change data type of a column
df_white = df_white.astype({"pH":int})
df_white.head(2)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ph_alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3,0.45,8.8,6,26.40
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3,0.49,9.5,6,31.35


## Pyspark Add a new column 

If the column name is already present, it will update the value of that column.

All the examples are only displaying output. Reassign result to a new dataframe 
for persistance

In [128]:
# withColumn to add a new column
# spark dataframes are immutable. Needs explicit reassignment to add column to the existing dataframe
df_red.withColumn('ph_alcohol',df_red.pH*df_red.alcohol).show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|        ph_alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|            32.994|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|31.360000000000003|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|      

In [129]:
# Use rounding on top of the newly created column

from pyspark.sql.functions import round
df_red.withColumn('ph_alcohol',round(df_red.pH*df_red.alcohol,2)).show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|ph_alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|     32.99|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|     31.36|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|     31.95|
+-------------+----------------+--

In [130]:
# Sample explicit reassignment. 
df_red = df_red.withColumn('ph_alcohol',round(df_red.pH*df_red.alcohol,2))
df_red.show(3)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|ph_alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|     32.99|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|     31.36|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5|     31.95|
+-------------+----------------+--

In [98]:
# Use withColumn along with col function
df_red.withColumn("quality_in_hundreds",col("quality")*10).show(4)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|quality_in_hundreds|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|               50.0|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|               50.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.65|    9.8|      5| 

In [187]:
# WithColumn is also used to change datatype of a column
df_red.withColumn("pH",col("pH").cast("Integer")).show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|  3|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968|  3|     0.68|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
only showing top 2 rows



# Pandas rename columns, Drop columns

In [155]:
# Syntax is df.rename(columns:{'existing_column_name':'new_column_name'})
df_white_renamed = df_white.rename(columns={'fixed acidity':'fixed_acidity'})
df_white_renamed.head(2)

,fixed_acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ph_alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6,26.40
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6,31.35


In [156]:
# inplace parameter allows pandas dataframe to return modified dataframe without need for reassignment
df_white_renamed.drop('ph_alcohol',axis=1,inplace=True)
df_white_renamed.head(2)

,fixed_acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6


In [157]:
# Drop multiple columns with pandas. Pass a list of column names 
df_white_renamed.drop(['pH','alcohol'],axis=1,inplace=True)
df_white_renamed.head(2)

,fixed_acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,0.45,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,0.49,6


## PySpark rename columns, Drop columns

In [150]:
# Syntax withColumnRenamed("existing_column_name","new_column_name")
df_red.withColumnRenamed("fixed acidity","fixed_acidity").show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed_acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 2 rows



In [149]:
# pass the column to drop, or a list of string name of the columns to drop
# drop requires reassignment for the effect to persist
df_red = df_red.drop("ph_alcohol")
df_red.show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 2 rows



In [143]:
# Use col function to select a single column
df_red.drop(col("quality")).show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
only showing top 2 rows



In [148]:
df_red.drop(df_red.quality).show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
only showing top 2 rows



In [158]:
# Drop multiple columns
# uses an array string as an argument
cols = ("fixed acidity","volatile acidity","citric acid")
df_red.drop(*cols).show(2)

+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5|
|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5|
+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
only showing top 2 rows



# Pandas GroupBy

In [164]:
# Use columnname directly for grouping by a single column. Pass a list to group by multiple columns for eg: ["density","pH"]
# Pandas displays the same count for all columns 
df_white.groupby("quality").count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,ph_alcohol
quality,,,,,,,,,,,,
3,20,20,20,20,20,20,20,20,20,20,20,20
4,163,163,163,163,163,163,163,163,163,163,163,163
5,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457
6,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198
7,880,880,880,880,880,880,880,880,880,880,880,880
8,175,175,175,175,175,175,175,175,175,175,175,175
9,5,5,5,5,5,5,5,5,5,5,5,5


In [174]:
# Select columns from the result of group by. Reset the group by index. Rename the selected column as count
df_white.groupby("quality").count()['pH'].reset_index().rename(columns={'pH':'count'})

,quality,count
0,3,20
1,4,163
2,5,1457
3,6,2198
4,7,880
5,8,175
6,9,5


## PySpark GroupBy

In [176]:
# pySpark returns the count of the grouped columns
# Unlike pandas, pyspark returns only one column for the aggregate operation. count in this case 
df_red.groupBy("quality").count().show()

+-------+-----+
|quality|count|
+-------+-----+
|      7|  199|
|      3|   10|
|      8|   18|
|      5|  681|
|      6|  638|
|      4|   53|
+-------+-----+



In [213]:
#Changing datatype for executing aggregate operations
# Tip: Such explicit casting can be avoided by using a schema with appropriate datatype while source file loading 
df_red_gp = df_red.withColumn("pH",col("pH").cast("Integer")).withColumn("density",col("density").cast("Float"))
df_red_gp.show(2)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|  3|     0.56|    9.4|      5|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968|  3|     0.68|    9.8|      5|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
only showing top 2 rows



In [214]:
#Verify pH and density columns are numeric and not string
df_red_gp.printSchema()

root
 |-- fixed acidity: string (nullable = true)
 |-- volatile acidity: string (nullable = true)
 |-- citric acid: string (nullable = true)
 |-- residual sugar: string (nullable = true)
 |-- chlorides: string (nullable = true)
 |-- free sulfur dioxide: string (nullable = true)
 |-- total sulfur dioxide: string (nullable = true)
 |-- density: float (nullable = true)
 |-- pH: integer (nullable = true)
 |-- sulphates: string (nullable = true)
 |-- alcohol: string (nullable = true)
 |-- quality: string (nullable = true)



In [195]:
# Returns mean pH for each quality distinct value
df_red_gp.groupBy("quality").avg("pH").show()
df_red_gp.groupBy("quality").max("pH").show() # Can use min function as well
df_red_gp.groupBy("quality").sum("pH").show() 

+-------+------------------+
|quality|           avg(pH)|
+-------+------------------+
|      7|2.9849246231155777|
|      3|               3.0|
|      8| 2.888888888888889|
|      5|2.9809104258443466|
|      6| 2.987460815047022|
|      4| 2.981132075471698|
+-------+------------------+

+-------+-------+
|quality|max(pH)|
+-------+-------+
|      7|      3|
|      3|      3|
|      8|      3|
|      5|      3|
|      6|      4|
|      4|      3|
+-------+-------+

+-------+-------+
|quality|sum(pH)|
+-------+-------+
|      7|    594|
|      3|     30|
|      8|     52|
|      5|   2030|
|      6|   1906|
|      4|    158|
+-------+-------+



In [208]:
# Use agg function to calculae more than one aggregate at a time.
# agg function will not work without function import.
from pyspark.sql.functions import sum,avg,max,min,mean,count

df_red_gp.groupBy("quality").agg(
    max("pH"),
    min("pH")).show()
# Rename columns using alias
df_red_gp.groupBy("quality").agg(
    max("pH").alias("max_pH"),
    min("pH").alias("min_pH")).show()


+-------+-------+-------+
|quality|max(pH)|min(pH)|
+-------+-------+-------+
|      7|      3|      2|
|      3|      3|      3|
|      8|      3|      2|
|      5|      3|      2|
|      6|      4|      2|
|      4|      3|      2|
+-------+-------+-------+

+-------+------+------+
|quality|max_pH|min_pH|
+-------+------+------+
|      7|     3|     2|
|      3|     3|     3|
|      8|     3|     2|
|      5|     3|     2|
|      6|     4|     2|
|      4|     3|     2|
+-------+------+------+



In [220]:
# Groupby on multiple columns
df_red_gp.groupBy("quality","pH")\
         .avg("density").show()

+-------+---+------------------+
|quality| pH|      avg(density)|
+-------+---+------------------+
|      7|  2|0.9996000329653422|
|      7|  3|0.9960507665361676|
|      6|  3|0.9966296972558141|
|      3|  3|0.9974640011787415|
|      4|  2|0.9995999932289124|
|      5|  3|0.9970651053026051|
|      8|  3|0.9950381256639957|
|      5|  2|0.9990830742395841|
|      8|  2|0.9966050088405609|
|      6|  2|0.9965059995651245|
|      6|  4| 0.992579996585846|
|      4|  3|0.9964836572225277|
+-------+---+------------------+



In [219]:
# Groupby on a subset of data using "where"
df_red_gp.groupBy("quality","pH") \
                       .avg("density") \
                       .where(col("quality")>7).show()

+-------+---+------------------+
|quality| pH|      avg(density)|
+-------+---+------------------+
|      8|  3|0.9950381256639957|
|      8|  2|0.9966050088405609|
+-------+---+------------------+



## Pandas Row level operatons such as Sorting, Filtering (row level subset), Appending new rows

In [225]:
# Sort dataframe by quality in descending order
df_white.sort_values(by='quality',ascending=False)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ph_alcohol
827,7.4,0.24,0.36,2.00,0.031,27.0,139.0,0.99055,3,0.48,12.5,9,41.000
1605,7.1,0.26,0.49,2.20,0.032,31.0,113.0,0.99030,3,0.42,12.9,9,43.473
876,6.9,0.36,0.34,4.20,0.018,57.0,119.0,0.98980,3,0.36,12.7,9,41.656
774,9.1,0.27,0.45,10.60,0.035,28.0,124.0,0.99700,3,0.46,10.4,9,33.280
820,6.6,0.36,0.29,1.60,0.021,24.0,85.0,0.98965,3,0.61,12.4,9,42.284
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,7.5,0.32,0.24,4.60,0.053,8.0,134.0,0.99580,3,0.50,9.1,3,28.574
2373,7.6,0.48,0.37,1.20,0.034,5.0,57.0,0.99256,3,0.54,10.4,3,31.720
251,8.5,0.26,0.21,16.20,0.074,41.0,197.0,0.99800,3,0.50,9.8,3,29.596
1688,6.7,0.25,0.26,1.55,0.041,118.5,216.0,0.99490,3,0.63,9.4,3,33.370


In [224]:
# Filter only rows with quality value 9
df_white[df_white['quality']==9]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ph_alcohol
774,9.1,0.27,0.45,10.6,0.035,28.0,124.0,0.99700,3,0.46,10.4,9,33.280
820,6.6,0.36,0.29,1.6,0.021,24.0,85.0,0.98965,3,0.61,12.4,9,42.284
827,7.4,0.24,0.36,2.0,0.031,27.0,139.0,0.99055,3,0.48,12.5,9,41.000
876,6.9,0.36,0.34,4.2,0.018,57.0,119.0,0.98980,3,0.36,12.7,9,41.656
1605,7.1,0.26,0.49,2.2,0.032,31.0,113.0,0.99030,3,0.42,12.9,9,43.473


130